# ブリックを実装する

これは，[このコード](https://github.com/nssuperx/irl334-research-srcs/blob/main/multivalue/play.py)とそれに付随するモジュールのipynb版です．

PyTorchを使用します．

## 1つのブリックを作る

何次元かの入力を受け取って，興奮したかしてないかを出力する素子を作ります．
* 入力するデータを用意
* ブリックの入力素子に入力
* 状態を選択
* 出力

の順で説明します．

### 入力するデータの用意

PyTorch DataLoaderを使用します．データセットはMNISTを使います．

In [5]:
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

datasets_path: str = "../pt_datasets"

train_dataset = datasets.MNIST(
    root=datasets_path,
    train=True,
    download=True,
    transform=ToTensor(),
)
test_dataset = datasets.MNIST(
    root=datasets_path,
    train=False,
    download=True,
    transform=ToTensor(),
)

MNIST_classes = datasets.MNIST.classes

In [ ]:
print(train_dataset)
print(MNIST_classes)

データセットを読み込みました．次はこれを扱いやすくします．PyTorchではDataLoaderを使います．

In [7]:
batch_size: int = 10

trainloader = DataLoader(train_dataset, batch_size, shuffle=True)
testloader = DataLoader(test_dataset, batch_size, shuffle=False)

In [ ]:
print(trainloader)